# Calculation of ionic strength and activity coefficients of aqueous species

In this tutorial, we explain how to access the ionic strength of the equilibrium state as well as clarify how it can be calculated manually by accessing specific properties of the chemical state. We also calculate the activity coefficients of aqueous species and solvent water.

First, we import all the required python packages:

In [1]:
from reaktoro import *
import numpy as np
from math import *

Define database, initialize phases with chemical editor, and create chemical system:

In [2]:
# Define the thermodynamic database
db =  SupcrtDatabase("supcrt98")

# Define the aqueous phase
aqueousphase = AqueousPhase(speciate("H O C Na Cl Ca"), exclude("organic"))
aqueousphase.setActivityModel(ActivityModelDebyeHuckel())

# Define the gaseous phase
gaseousphase = GaseousPhase("H2O(g)")

# Define the chemical system
system = ChemicalSystem(db, aqueousphase, gaseousphase)

# Define the chemical prperties
properties = ChemicalProps(system)

Set thermodynamic conditions:

In [3]:
T = 25 # in celsius
P = 1 # in bar

First problem simulates mixing sodium chlorite with water:

In [4]:
state1 = ChemicalState(system)
state1.temperature(T, "celsius")
state1.pressure(P, "bar")
state1.add("H2O(aq)", 1.0, "kg")
state1.add("NaCl(aq)", 1.0, "mol")

The second one demonstrates experiment of mixing water with CaCl<sub>2</sub>:

In [5]:
state2 = ChemicalState(system)
state2.temperature(T, "celsius")
state2.pressure(P, "bar")
state2.add("H2O(aq)", 1.0, "kg")
state2.add("CaCl2(aq)", 1, "mol")

Equilibration of the water mixed 1 mol of sodium chlorite and water mixed with 1 mol of CaCl<sub>2</sub> results into the following two states:

In [6]:
res1 = equilibrate(state1)
res2 = equilibrate(state2)

To evaluate the ionic strength, we need to define a corresponding function `ionic_strength()`

In [7]:
aprops1 = AqueousProps(state1)
aprops2 = AqueousProps(state2)

I1 = float(aprops1.ionicStrength())
I2 = float(aprops2.ionicStrength())

print(f"Ionic strength of state1 is {I1:f} molal")
print(f"Ionic strength of state2 is {I2:f} molal")

Ionic strength of state1 is 0.947121 molal
Ionic strength of state2 is 2.094351 molal


We see that the ionic strength of the second mix is higher, which can be explained by the fact that CaCl<sub>2</sub> contains two ions of Cl<sup>-</sup>.

## Calculating the ionic strength

Below, we explain, which information one needs to fetch from chemical state to be able to calculate ionic strength. First, let us fix the molality of 1 kg of solvent water (where 18.0154 * 1e-3 kg/mol is a molar mass of water):

In [8]:
mw_h2o = 1 / 18.0154 / 1e-3
print(f"mw_h2o = {mw_h2o:f} molal")

mw_h2o = 55.508065 molal


Next we collect the list of species and their amounts in the chemical system:

In [9]:
species = system.species()
n2 = state2.speciesAmounts().asarray()

Since out of two phases (aqueous and gaseous) we need to consider only species from the aqueous phase, we fetch indices of the aqueous species.

In [10]:
phases = system.phases()
species = system.species()

aq_species = phases.get("AqueousPhase").species()
gas_species = phases.get("GaseousPhase").species()

indx_aqueous_species = [species.findWithName(s.name()) for s in aq_species]
indx_gas_species = [species.findWithName(s.name()) for s in gas_species]
indx_all_species = indx_aqueous_species + indx_gas_species

print(f"Indices of aq. species:", indx_aqueous_species)
print(f"Indices of gas. species:", indx_gas_species)
print(f"Indices of all species:", indx_all_species)

Indices of aq. species: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
Indices of gas. species: [33]
Indices of all species: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]


We see that the difference between these two lists is only in the last index.

Next, out the list `species` (with all species), we collect lists of only aqueous species, their amounts, names, and corresponding charges:

In [11]:
species_aq = [species[i] for i in indx_aqueous_species]
n2_aq = [n2[i] for i in indx_aqueous_species]
names_aq = [species.name() for species in species_aq]
charge_aq = [float(species.charge()) for species in species_aq]

Amount of the water is obtained by:

In [12]:
n_h2o = float(state2.speciesAmount("H2O(aq)"))

Next, we calculate the molalities of aqueous species and print their names, charges, and molalities:

In [13]:
m_aq = mw_h2o * np.divide(n2_aq, n_h2o)
print(f"Species               : charge, molalities")
for name, Z, m in zip(names_aq, charge_aq, m_aq):
    print(f"{name:21s} : {Z:6.0f}, {m:6.2e} molal")

Species               : charge, molalities
H2O(aq)               :      0, 5.55e+01 molal
2-Hydroxynonanoate-   :     -1, 1.00e-16 molal
2-Hydroxynonanoic(aq) :      0, 1.00e-16 molal
CaOH+                 :      1, 2.51e-07 molal
CO(aq)                :      0, 1.00e-16 molal
CO2(aq)               :      0, 1.00e-16 molal
CO3-2                 :     -2, 1.00e-16 molal
Ca(HCO3)+             :      1, 1.00e-16 molal
Ca+2                  :      2, 5.56e-01 molal
CaCl+                 :      1, 4.25e-01 molal
CaCl2(aq)             :      0, 1.84e-02 molal
Cl-                   :     -1, 1.54e+00 molal
HClO(aq)              :      0, 1.00e-16 molal
ClO-                  :     -1, 1.00e-16 molal
ClO2-                 :     -1, 1.00e-16 molal
ClO3-                 :     -1, 1.00e-16 molal
ClO4-                 :     -1, 1.00e-16 molal
H+                    :      1, 2.66e-07 molal
H2(aq)                :      0, 1.00e-16 molal
HCO3-                 :     -1, 1.00e-16 molal
HO2-             

The ionic strength can be calculated by:

In [14]:
I2 = 1/2 * sum([m * Z**2 for m, Z in zip(m_aq, charge_aq)])
print(f"Ionic strength of state2 is {I2:f} molal (calculated manually)")

Ionic strength of state2 is 2.094336 molal (calculated manually)


## Calculating the activity coefficients for aqueous ionic species (Davis model)

Calculating and outputting the activity coefficients for aqueous ionic species is done by:

In [15]:
A_gamma = 0.5095
gammas = [10**(-A_gamma * z**2 * (sqrt(I2) / (1 + sqrt(I2)) - 0.3 * I2)) for z in charge_aq]
print(f"              Species : Activity coefficients")
for name, gamma in zip(names_aq, gammas):
    print(f"{name:>21} : {gamma:2.4f}")

              Species : Activity coefficients
              H2O(aq) : 1.0000
  2-Hydroxynonanoate- : 1.0443
2-Hydroxynonanoic(aq) : 1.0000
                CaOH+ : 1.0443
               CO(aq) : 1.0000
              CO2(aq) : 1.0000
                CO3-2 : 1.1892
            Ca(HCO3)+ : 1.0443
                 Ca+2 : 1.1892
                CaCl+ : 1.0443
            CaCl2(aq) : 1.0000
                  Cl- : 1.0443
             HClO(aq) : 1.0000
                 ClO- : 1.0443
                ClO2- : 1.0443
                ClO3- : 1.0443
                ClO4- : 1.0443
                   H+ : 1.0443
               H2(aq) : 1.0000
                HCO3- : 1.0443
                 HO2- : 1.0443
           Nonanoate- : 1.0443
    Nonanoic-Acid(aq) : 1.0000
                  Na+ : 1.0443
             NaCl(aq) : 1.0000
             NaOH(aq) : 1.0000
               O2(aq) : 1.0000
                  OH- : 1.0443
          Nonanal(aq) : 1.0000
             H2O2(aq) : 1.0000
            HClO2(aq) : 

We see that many of the activity coefficients are away from $\gamma_i$ = 1 (which corresponds to an ideal solution).

## Calculating activity of the water solvent

To calculate the activity of the water solvent, we need fractions of the species, which are stored in the class [ChemicalProperties](https://reaktoro.org/cpp/classReaktoro_1_1ChemicalProperties.html) accessed from chemical state by the method `properties()`.

In [16]:
properties.update(state2)
fractions = properties.speciesMoleFractions().asarray()

Let us output only those species that have fractions bigger than machine precision (fetched from the field `epsilon` of a class [EquilibriumOptions](https://reaktoro.org/api/struct_reaktoro_1_1_equilibrium_options.html), a default lower bound for the amounts of the species):

In [17]:
machine_precision = EquilibriumOptions().epsilon
print(f"   Species : Mole fractions")
for name, x in zip(names_aq, fractions):
    if x > machine_precision:
        print(f"{name:>10} : {x:6.4e}")

   Species : Mole fractions
   H2O(aq) : 9.5628e-01
     CaOH+ : 4.3239e-09
      Ca+2 : 9.5846e-03
     CaCl+ : 7.3268e-03
 CaCl2(aq) : 3.1621e-04
       Cl- : 2.6496e-02
        H+ : 4.5788e-09
       OH- : 6.0640e-10
   HCl(aq) : 3.5150e-10


We see that solvent water possesses the biggest fraction as well as CaCl<sub>2</sub>(aq) and ions Ca<sup>2+</sup>, CaCl<sup>+</sup>, and Cl<sup>-</sup> (caused by addition of CaCl<sub>2</sub> to the water). The fraction of solvent water can be accessed via index of this species:

In [18]:
indx_h2o = species.findWithName("H2O(aq)")
x_h2o = fractions[indx_h2o]
print(f"Index of the water solvent is {indx_h2o}")
print(f"Fraction of the water solvent is {x_h2o:6.4f}")

Index of the water solvent is 0
Fraction of the water solvent is 0.9563


Finally, we calculate activity coefficient of solvent water by:

In [19]:
ln10 = np.log(10.0)
sqrtI2 = np.sqrt(I2)
gamma_h2o_davis = exp(ln10/55.5084*A_gamma*(2*(I2 + 2*sqrtI2)/(1 + sqrtI2) - 4 * np.log(1 + sqrtI2) - 0.3 * I2**2)
                      - (1 - x_h2o)/x_h2o)
gamma_h2o_ideal = exp(- (1 - x_h2o)/x_h2o)
print(f"Activity coefficient of water solvent (Davis model) is {gamma_h2o_davis:6.4f}")
print(f"Activity coefficient of water solvent (ideal model) is {gamma_h2o_ideal:6.4f}")

Activity coefficient of water solvent (Davis model) is 0.9389
Activity coefficient of water solvent (ideal model) is 0.9553


## Demonstration of Coulomb’s law

According to Coulomb’s law, the activity coefficient decreases as the concentration increases because the electrostatic forces become stronger as the ions approach. Thus, for more concentrated solutions, the repulsion effect seems to dominate. Let us demonstrate how it can be seen in Reaktoro simulations. First, we access the activity coefficients of the `state2` via its properties obtained earlier:

In [20]:
gamma_1_mol = [np.exp(float(properties.speciesActivityCoefficientLn(i))) for i in indx_all_species]

In [21]:
state2.add("CaCl2(aq)", 2, "mol")
res2 = equilibrate(state2)
properties.update(state2)
gamma_2_mol = [np.exp(float(properties.speciesActivityCoefficientLn(i))) for i in indx_all_species]

In [22]:
print(f"Species with decreased activity coeffs. after adding more CaCl2 to the water:")
for name, gamma1, gamma2 in zip(names_aq, gamma_1_mol, gamma_2_mol):
    if gamma_1_mol > gamma_2_mol:
        print(f"{name:21s} : {gamma1:6.4e} -> {gamma2:6.4e}")

Species with decreased activity coeffs. after adding more CaCl2 to the water:
H2O(aq)               : 3.6824e-01 -> 9.0789e-02
2-Hydroxynonanoate-   : 1.7794e-01 -> 9.3451e-02
2-Hydroxynonanoic(aq) : 1.6404e+00 -> 2.5423e+00
CaOH+                 : 1.7794e-01 -> 9.3451e-02
CO(aq)                : 1.6404e+00 -> 2.5423e+00
CO2(aq)               : 1.6404e+00 -> 2.5423e+00
CO3-2                 : 1.4711e-01 -> 1.2582e-01
Ca(HCO3)+             : 6.4176e-01 -> 6.2075e-01
Ca+2                  : 2.9870e-01 -> 5.1651e-01
CaCl+                 : 1.7794e-01 -> 9.3451e-02
CaCl2(aq)             : 1.6404e+00 -> 2.5423e+00
Cl-                   : 5.8061e-01 -> 5.8390e-01
HClO(aq)              : 1.6404e+00 -> 2.5423e+00
ClO-                  : 1.7794e-01 -> 9.3451e-02
ClO2-                 : 1.7794e-01 -> 9.3451e-02
ClO3-                 : 1.7794e-01 -> 9.3451e-02
ClO4-                 : 1.7794e-01 -> 9.3451e-02
H+                    : 7.2369e-01 -> 7.1125e-01
H2(aq)                : 1.6404e+00 -> 2.